In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.cluster import KMeans
from sklearn.pipeline import make_pipeline
from yellowbrick.cluster import KElbowVisualizer
from sklearn import set_config
from sklearn.decomposition import PCA
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")



In [2]:
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 72
plt.style.use('ggplot')
#pd.options.display.float_format = '{:.0f}'.format
#np.random.seed(0)
#np.set_printoptions(precision=2, linewidth=100)
#warnings.filterwarnings(action='ignore')
sns.set_context('notebook')

In [3]:
df = pd.read_csv('marketing_campaign.csv',delimiter = '\t',parse_dates=['Dt_Customer'])

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   2240 non-null   int64         
 1   Year_Birth           2240 non-null   int64         
 2   Education            2240 non-null   object        
 3   Marital_Status       2240 non-null   object        
 4   Income               2216 non-null   float64       
 5   Kidhome              2240 non-null   int64         
 6   Teenhome             2240 non-null   int64         
 7   Dt_Customer          2240 non-null   datetime64[ns]
 8   Recency              2240 non-null   int64         
 9   MntWines             2240 non-null   int64         
 10  MntFruits            2240 non-null   int64         
 11  MntMeatProducts      2240 non-null   int64         
 12  MntFishProducts      2240 non-null   int64         
 13  MntSweetProducts     2240 non-nul

In [5]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,2012-04-09,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,2014-08-03,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,2013-08-21,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,2014-10-02,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,2014-01-19,94,173,...,5,0,0,0,0,0,0,3,11,0


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID                   2240 non-null   int64         
 1   Year_Birth           2240 non-null   int64         
 2   Education            2240 non-null   object        
 3   Marital_Status       2240 non-null   object        
 4   Income               2216 non-null   float64       
 5   Kidhome              2240 non-null   int64         
 6   Teenhome             2240 non-null   int64         
 7   Dt_Customer          2240 non-null   datetime64[ns]
 8   Recency              2240 non-null   int64         
 9   MntWines             2240 non-null   int64         
 10  MntFruits            2240 non-null   int64         
 11  MntMeatProducts      2240 non-null   int64         
 12  MntFishProducts      2240 non-null   int64         
 13  MntSweetProducts     2240 non-nul

In [7]:
df.shape

(2240, 29)

In [8]:
df.describe()

,ID,Year_Birth,Income,Kidhome,Teenhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
count,2240.000000,2240.000000,2216.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,...,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.000000,2240.0,2240.0,2240.000000
mean,5592.159821,1968.805804,52247.251354,0.444196,0.506250,49.109375,303.935714,26.302232,166.950000,37.525446,...,5.316518,0.072768,0.074554,0.072768,0.064286,0.013393,0.009375,3.0,11.0,0.149107
std,3246.662198,11.984069,25173.076661,0.538398,0.544538,28.962453,336.597393,39.773434,225.715373,54.628979,...,2.426645,0.259813,0.262728,0.259813,0.245316,0.114976,0.096391,0.0,0.0,0.356274
min,0.000000,1893.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
25%,2828.250000,1959.000000,35303.000000,0.000000,0.000000,24.000000,23.750000,1.000000,16.000000,3.000000,...,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
50%,5458.500000,1970.000000,51381.500000,0.000000,0.000000,49.000000,173.500000,8.000000,67.000000,12.000000,...,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
75%,8427.750000,1977.000000,68522.000000,1.000000,1.000000,74.000000,504.250000,33.000000,232.000000,50.000000,...,7.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.0,11.0,0.000000
max,11191.000000,1996.000000,666666.000000,2.000000,2.000000,99.000000,1493.000000,199.000000,1725.000000,259.000000,...,20.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,3.0,11.0,1.000000


In [9]:
df = df[df['Income'] < 200000]  

In [10]:
df.isna().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64

In [11]:
df.columns[df.dtypes == 'O']

Index(['Education', 'Marital_Status'], dtype='object')

In [12]:
df.dropna(inplace = True)

In [13]:
df.isna().sum()

ID                     0
Year_Birth             0
Education              0
Marital_Status         0
Income                 0
Kidhome                0
Teenhome               0
Dt_Customer            0
Recency                0
MntWines               0
MntFruits              0
MntMeatProducts        0
MntFishProducts        0
MntSweetProducts       0
MntGoldProds           0
NumDealsPurchases      0
NumWebPurchases        0
NumCatalogPurchases    0
NumStorePurchases      0
NumWebVisitsMonth      0
AcceptedCmp3           0
AcceptedCmp4           0
AcceptedCmp5           0
AcceptedCmp1           0
AcceptedCmp2           0
Complain               0
Z_CostContact          0
Z_Revenue              0
Response               0
dtype: int64

In [14]:
df[df.duplicated('Dt_Customer')]

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
11,387,1976,Basic,Married,7500.0,0,0,2012-11-13,59,6,...,8,0,0,0,0,0,0,3,11,0
13,8180,1952,Master,Divorced,59354.0,1,1,2013-11-15,53,233,...,6,0,0,0,0,0,0,3,11,0
32,5710,1970,Graduation,Together,40548.0,0,1,2012-10-10,31,110,...,5,0,1,0,0,0,0,3,11,0
42,8430,1957,Graduation,Together,21994.0,0,1,2012-12-24,4,9,...,5,0,0,0,0,0,0,3,11,0
52,1331,1977,Graduation,Single,35790.0,1,0,2013-02-01,54,12,...,7,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2235,10870,1967,Graduation,Married,61223.0,0,1,2013-06-13,46,709,...,5,0,0,0,0,0,0,3,11,0
2236,4001,1946,PhD,Together,64014.0,2,1,2014-10-06,56,406,...,7,0,0,0,1,0,0,3,11,0
2237,7270,1981,Graduation,Divorced,56981.0,0,0,2014-01-25,91,908,...,6,0,1,0,0,0,0,3,11,0
2238,8235,1956,Master,Together,69245.0,0,1,2014-01-24,8,428,...,3,0,0,0,0,0,0,3,11,0


In [15]:
df.nunique()

ID                     2215
Year_Birth               59
Education                 5
Marital_Status            8
Income                 1973
Kidhome                   3
Teenhome                  3
Dt_Customer             662
Recency                 100
MntWines                776
MntFruits               158
MntMeatProducts         554
MntFishProducts         182
MntSweetProducts        176
MntGoldProds            212
NumDealsPurchases        15
NumWebPurchases          15
NumCatalogPurchases      14
NumStorePurchases        14
NumWebVisitsMonth        16
AcceptedCmp3              2
AcceptedCmp4              2
AcceptedCmp5              2
AcceptedCmp1              2
AcceptedCmp2              2
Complain                  2
Z_CostContact             1
Z_Revenue                 1
Response                  2
dtype: int64

In [16]:
# plt.figure(figsize=(15,18))
# for i , col in enumerate(df.columns):
#     plt.subplot(10,3,i+1)
#     g = sns.countplot(x=col, data=df)
#     g.set(title=col)
# plt.tight_layout()


# plt.show()

In [17]:
# plt.figure(figsize=(15,3))
# for i , col in enumerate(['Year_Birth','Income','Recency']):
    
#     if col not in df.columns[df.dtypes == 'O']:
#         plt.subplot(1,3,i+1)

#         g = sns.boxplot(x=col, data=df)
#         g.set(title=col)
# plt.tight_layout()

# plt.show()

In [18]:
# plt.figure(figsize=(15,18))
# for i , col in enumerate(df.columns[df.nunique() < 10]):
    
    
#     plt.subplot(5,3,i+1)
#     g = sns.boxplot(x=col,y='Income', data=df)
#     g.set(title=col)
#     plt.xticks(rotation = 45)
# plt.tight_layout()

# plt.show()

## Feature Engineering

In [19]:
df.nunique()

ID                     2215
Year_Birth               59
Education                 5
Marital_Status            8
Income                 1973
Kidhome                   3
Teenhome                  3
Dt_Customer             662
Recency                 100
MntWines                776
MntFruits               158
MntMeatProducts         554
MntFishProducts         182
MntSweetProducts        176
MntGoldProds            212
NumDealsPurchases        15
NumWebPurchases          15
NumCatalogPurchases      14
NumStorePurchases        14
NumWebVisitsMonth        16
AcceptedCmp3              2
AcceptedCmp4              2
AcceptedCmp5              2
AcceptedCmp1              2
AcceptedCmp2              2
Complain                  2
Z_CostContact             1
Z_Revenue                 1
Response                  2
dtype: int64

In [20]:
df['Education'].value_counts()

Graduation    1115
PhD            481
Master         365
2n Cycle       200
Basic           54
Name: Education, dtype: int64

In [21]:
df['Education'] = df['Education'].replace({'Graduation' : 'Graduate', 
                                           'PhD': 'Postgrad', 'Master':'Postgrad',
                                           '2n Cycle':'Undergrad', 'Basic':'Undergrad'})

In [22]:
df['Marital_Status'].value_counts()

Married     857
Together    572
Single      471
Divorced    232
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64

In [23]:
df['Living_With'] = df['Marital_Status'].replace({'Married':'Partner', 'Together':'Partner',
                                                  'Single':'Alone','Divorced':'Alone','Widow':'Alone',
                                                  'Absurd':'Alone', 'YOLO':'Alone'})

In [24]:
df['Living_With'].value_counts()

Partner    1429
Alone       786
Name: Living_With, dtype: int64

In [25]:
# df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'])

In [26]:
df['Age'] = df['Dt_Customer'].dt.year - df['Year_Birth']

In [27]:
df['Age']

0       55
1       60
2       48
3       30
4       33
        ..
2235    46
2236    68
2237    33
2238    58
2239    58
Name: Age, Length: 2215, dtype: int64

In [28]:
df['Children'] = df['Kidhome'] + df['Teenhome']
df['Children'].value_counts()

1    1116
0     633
2     416
3      50
Name: Children, dtype: int64

In [29]:
df['spent'] = df['MntWines'] + df['MntFruits'] + df['MntMeatProducts'] + df['MntFishProducts'] + \
                df['MntSweetProducts'] + df['MntGoldProds' ]

In [31]:
ds = df.drop(columns = ['AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','AcceptedCmp1','AcceptedCmp2',
                        'Complain','Z_CostContact','Z_Revenue','Response','ID','Marital_Status',
                        'Year_Birth','Dt_Customer','Kidhome','Teenhome','spent'])

In [32]:
ds.shape

(2215, 17)

In [33]:
ds['Is_parent'] = np.where(ds.Children> 0, 1, 0)

In [34]:
ds.head()

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Living_With,Age,Children,Is_parent
0,Graduate,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,Alone,55,0,0
1,Graduate,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,Alone,60,2,1
2,Graduate,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,Partner,48,0,0
3,Graduate,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,Partner,30,1,1
4,Postgrad,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,Partner,33,1,1


In [35]:
ds.nunique()

Education                 3
Income                 1973
Recency                 100
MntWines                776
MntFruits               158
MntMeatProducts         554
MntFishProducts         182
MntSweetProducts        176
MntGoldProds            212
NumDealsPurchases        15
NumWebPurchases          15
NumCatalogPurchases      14
NumStorePurchases        14
NumWebVisitsMonth        16
Living_With               2
Age                      61
Children                  4
Is_parent                 2
dtype: int64

In [36]:
oh = OneHotEncoder(drop='if_binary',sparse=False)
oh.fit_transform(ds[['Education']])

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [37]:
ssc = StandardScaler()

In [42]:
dsnoncat = [i for i in ds.columns if i not in ['Education', 'Living_With','Is_parent']]

In [43]:
ct = ColumnTransformer([("onehotcategory", oh, ['Education', 'Living_With']),
     ("scaler", ssc, dsnoncat)])

In [45]:
# X = ct.fit_transform(ds)

In [47]:
# X.shape

(2215, 19)

In [73]:
pipe = make_pipeline(ct , PCA(n_components = 3),KMeans(n_clusters=4))

In [74]:
pipe.fit_predict(ds)

array([3, 0, 1, ..., 1, 1, 0], dtype=int32)

In [59]:
ds['label'] = pipe.fit_predict(ds)

In [60]:
ds.head()

,Education,Income,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Living_With,Age,Children,Is_parent,label
0,Graduate,58138.0,58,635,88,546,172,88,88,3,8,10,4,7,Alone,55,0,0,3
1,Graduate,46344.0,38,11,1,6,2,1,6,2,1,1,2,5,Alone,60,2,1,0
2,Graduate,71613.0,26,426,49,127,111,21,42,1,8,2,10,4,Partner,48,0,0,1
3,Graduate,26646.0,26,11,4,20,10,3,5,2,2,0,4,6,Partner,30,1,1,0
4,Postgrad,58293.0,94,173,43,118,46,27,15,5,5,3,6,5,Partner,33,1,1,2


In [57]:
set_config(display='diagram')
# displays HTML representation in a jupyter context
pipe

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('onehotcategory',
                                                  OneHotEncoder(drop='if_binary',
                                                                sparse=False),
                                                  ['Education', 'Living_With']),
                                                 ('scaler', StandardScaler(),
                                                  ['Income', 'Recency',
                                                   'MntWines', 'MntFruits',
                                                   'MntMeatProducts',
                                                   'MntFishProducts',
                                                   'MntSweetProducts',
                                                   'MntGoldProds',
                                                   'NumDealsPurchases',
                                                   'NumWebPurchases',
                                                   'NumCatalogPurchases',
                                                   'NumStorePurchases',
                                                   'NumWebVisitsMonth', 'Age',
                                                   'Children'])])),
                ('pca', PCA(n_components=3)),
                ('kmeans', KMeans(n_clusters=4))])